In [1]:
import openai
import os
org_key_file = 'org_key'
api_key_file = 'api_key'
openai.organization = open(org_key_file).read().strip()
openai.api_key = open(api_key_file).read().strip()

In [2]:
from helpers import *
from sklearn.model_selection import train_test_split

path = "data/ner_annotations.json"
sentences, all_labels, label_list, label_dict = load_data(path)

X_train, X_rest, y_train, y_rest = train_test_split(
    sentences,
    all_labels,
    test_size=0.66666,
    random_state=1
    )

X_test, X_val, y_test, y_val = train_test_split(
    X_rest,
    y_rest,
    test_size=0.5,
    random_state=1
    )

In [3]:
import torch
import pandas as pd

with open("./20230811-135010_train_data_losses.pt", 'rb') as f:
    losses = torch.load(f)

thresh_low = 1.8
thresh_high = 2.0
sent_label_pairs = [l[1] for l in losses
                    if l[0] > thresh_low and l[0] <= thresh_high]

print(len(sent_label_pairs))
for e in sent_label_pairs[:3]:
    print(e[0])
    print(e[1])
    print()

69
['Catalytic', 'consequences', 'of', 'micropore', 'topology', ',', 'mesoporosity', ',', 'and', 'acidity', 'on', 'the', 'hydrolysis', 'of', 'sucrose', 'over', 'zeolite', 'catalysts', '.']
['null', 'null', 'null', 'null', 'property-type', 'null', 'property-type', 'null', 'null', 'property-type', 'null', 'null', 'meta', 'null', 'null', 'null', 'null', 'null', 'null']

['Organic', 'solvents', 'and', 'other', 'chemicals', 'used', 'were', 'either', 'HPLC', 'or', 'analytical', 'grade', 'from', 'Merck', ',', 'Fluka', ',', 'Panreac', 'and', 'Aldrich', '.']
['material-descriptor', 'unspecified-material', 'null', 'null', 'unspecified-material', 'operation', 'null', 'null', 'amount-misc', 'null', 'amount-misc', 'amount-misc', 'null', 'brand', 'null', 'brand', 'null', 'brand', 'null', 'brand', 'null']

['All', 'zeolite', 'samples', 'in', 'their', 'NH4+', '-', 'form', 'were', 'treated', 'in', 'dry', 'air', '(', '100', 'mL', 'min-1', ',', 'ultrapure', ',', 'Airgas', ')', 'by', 'increasing', 'the', 

In [4]:
responses = []
understanding = []
log = {
    "n_responses": 0,
    "completion_tokens": 0,
    "prompt_tokens": 0,
    "total_tokens": 0,
    "errors" : []
    }

In [5]:
import torch
import time

def save_stuff(responses, log):
    file_id = time.strftime("%Y%m%d-%H%M%S") + '_'
    with open(f"./augmentations/{file_id}responses.pt", 'wb') as f:
        torch.save(responses, f)

    with open(f"./logs/{file_id}log.pt", 'wb') as f:
        torch.save(log, f)

In [6]:
system_prompt = "Your job is to assist with data augmentation in the material synthesis domain."
augmentation_prompt = "Write as many new examples as possible in the style of the given examples. Output only the new examples without additional explanation. Tag all entities in an XML-style."
descriptions = {
    'apparatus-unit': "A unit of measurement used to quantify a capacity or dimension of a synthesis-apparatus. For example, 'kV' can be the unit of measurement for the accelerating voltage of an X-ray measurement apparatus or 'mL' as the unit quantifying the capacity of an autoclave",
    'characterization-apparatus': "A tool or instrument used for characterizing or analyzing the properties or characteristics of a material. For example, an 'X-ray measurement' apparatus used for elemental mapping or 'XRF' indicating that X-ray fluorescence spectroscopy was used to analyze the chemical composition of a precursor",
    'apparatus-property-type': "A specific type of property or characteristic associated with an apparatus. For example, 'capacity' is a property type associated with an autoclave or the 'amplitude' representing the intensity or strength of a sonication.",
    'condition-type': "This entity refers to a specific condition or parameter related to an experiment or process. For example, 'applied voltage' is a condition type associated with the electrospinning process.",
    'number': "Numerical values. For example, words like '14.7', '1', '2.5', and '10000' are number entities as representing specific quantities or measurements.",
    'unspecified-material': "A material that is not explicitly specified or described. For example, 'solution' and 'mixture' refer to unspecified materials.",
    'material': "A substance or matter that can undergo chemical reactions or be used to create products. For example, 'MnP' is a specific material used for membranes.",
    'synthesis-apparatus': "The equipment used for synthesis or production of a material. For example, 'syringe pump' and 'jet' are synthesis apparatuses used in the electrospinning process.",
    'material-descriptor': "Descriptions or characteristics of a substance or material. Words like 'green' and 'poly(vinyl alcohol)' describe the nature of the materials involved. Or 'aqueous solution' can be a material descriptor for an unspecified-material.",
    'condition-unit': "Units of measurement for specific conditions or processes. For example, 'K' (Kelvin) and 'h' (hour) describe the temperature and duration of the sintering process.",
    'operation': "Actions or processes performed on the entities or substances. For example, words like 'dissolved', 'stirring', and 'mixed' are all examples of operations carried out during the synthesis or reaction processes.",
    'apparatus-descriptor': "Describes the characteristics or properties of a specific apparatus or equipment. For example, '200-mesh' specifies the mesh size of a screen used during the sieving operation.",
    'precursor': "A substance or compound that is used as a starting material or ingredient in a chemical reaction or synthesis process. For example, words like 'manganese acetate tetrahydrate' and 'oleic acid' can be precursors if they are the starting materials for a chemical reaction or process.",
    'condition-misc': "Miscellaneous conditions or factors that are relevant to the context. For example, 'magnetic' describes the condition under which the stirring is performed and 'vigorous' and 'homogeneous' are mentioned as miscellaneous conditions describing the stirring and dispersion of precursors.",
    'property-misc': "Miscellaneous properties or characteristics of a substance or material. For example, 'cobalt-based' and 'catalyst' describe the nature of a reference material such as 'Au / AC'.",
    'target': "The desired or intended outcome or product of a process or reaction. For example, 'Au / AC', 'Co / SiO2' and 'zinc oxide' represent target entities if they are the desired material to be prepared. Targets are often mentioned in the title or headline of a recipe.",
    'amount-unit': "The unit of measurement for a quantity. For example, words like 'g', 'ml', 'L', 'μg mL-1', 'wt %', and 'MW' are specifying the unit in which a quantity is measured.",
    'reference': "Citations of a source that provides additional information or support for a statement or procedure. For example, 'Biella et al. 12' or '[12]' represent reference entities linking to a source.",
    'solvent': "A substance in which another substance is dissolved or mixed, typically in liquid form. 'methanol', 'water', 'H2O' or 'HF' are examples of solvents.",
    'gas': "Represents a type of gas used in a chemical process or reaction. For example, 'argon' is the gas used during a heating process.",
    'property-type': "A specific characteristic or quality of a material that can be observed. For example, 'diameter' and 'thickness' are used to describe the dimensions of pellets, while 'morphology' refers to the shape or form of nanostructures.",
    'property-unit': "Represents the unit of measurement used to quantify a property. For example, 'mm' (millimeter) is used to measure the diameter and thickness of pellets.",
    'brand': "The brand or manufacturer of materials or equipment. For example, 'Bunkokeiki' is the brand of a characterization apparatus used for measurements.",
    'meta': "A word or phrase that provides additional information or context about a process or experiment. For example, 'simple hydrothermal method' is meta-information on the preparation method for ferromagnetic nano-sized material.",
    'nonrecipe-material': 'This entity type refers to materials that are not part of a specified recipe or procedure. For example, "toluene" is tagged as nonrecipe-material because it is mentioned as a material added without being part of a specific recipe.',
    'amount-misc': 'This entity type refers to miscellaneous mentions of quantities or amounts. For example, "stoichiometric amount" and "required amount" are tagged as amount-misc because they indicate unspecified quantities of starting materials and solvent, respectively.'
}

In [7]:
from IPython.display import clear_output

save_every = 20
random.shuffle(sent_label_pairs)
for i in range(len(sent_label_pairs)-1):
    sample1 = sent_label_pairs[i]
    sample2 = sent_label_pairs[i+1]
    context_prompt, entity_set = create_context_prompt(
        sample1[0],
        sample1[1],
        sample2[0],
        sample2[1]
    )
    entity_relations = get_descriptions(entity_set, descriptions)

    try:
        augmentations = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role" : "system", "content" : system_prompt},
                {"role" : "user", "content" : context_prompt},
                {"role" : "assistant", "content" : entity_relations},
                {"role" : "user", "content" : augmentation_prompt},
                ],
            max_tokens=1000,
            temperature=0.7,
            n=3
            )
        clear_output()
        print(context_prompt)
        print(entity_relations)
        print(augmentations["choices"][0]["message"]["content"])
        responses.append(augmentations)
        log["n_responses"] += 1
        log["completion_tokens"] += augmentations["usage"]["completion_tokens"]
        log["prompt_tokens"] += augmentations["usage"]["prompt_tokens"]
        log["total_tokens"] += augmentations["usage"]["total_tokens"]
        
    except Exception as e:
        print("Request failed with Exception:", e)
        log["errors"].append(e)

    if i%save_every == (save_every-1):
        save_stuff(responses, log)
    

You are given this set of entities:
property-misc
operation
target

The entities are tagged in the examples in an XML-style.

Example 1: <operation> Calcd.

Example 2: A <property-misc> Microsized Cagelike </property-misc> <target> Sulfur / Carbon </target> <property-misc> Composite </property-misc> for a <property-misc> Lithium / Sulfur Battery </property-misc> with Excellent Performance.

Give a definition of each entity type and explain why the words from the examples belong to this entity.

property-misc : Miscellaneous properties or characteristics of a substance or material. For example, 'cobalt-based' and 'catalyst' describe the nature of a reference material such as 'Au / AC'.,
operation : Actions or processes performed on the entities or substances. For example, words like 'dissolved', 'stirring', and 'mixed' are all examples of operations carried out during the synthesis or reaction processes.,
target : The desired or intended outcome or product of a process or reaction. For 

In [8]:
save_stuff(responses, log)

In [9]:
aug_sentences, aug_labels = extract_augmentations(responses, label_list)
print(len())
for s in zip(aug_sentences, aug_labels):
    print(s[0])
    print(s[1])
    print()

TypeError: len() takes exactly one argument (0 given)